<a href="https://colab.research.google.com/github/AndreasAvgou/Liver-Disease-Prediction/blob/main/Liver_Disease_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import warnings
import matplotlib.pyplot as plt
plt.style.use('ggplot')

plt.rcParams['figure.dpi'] = 300

import pickle

from imblearn.pipeline import Pipeline


from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2,f_classif

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import svm, tree

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix,multilabel_confusion_matrix,classification_report
from sklearn.metrics import precision_recall_curve,plot_precision_recall_curve,plot_roc_curve
from sklearn.metrics import accuracy_score,f1_score

from tqdm.notebook import tqdm
from google.colab import drive
from google.colab import drive

from tqdm.notebook import tqdm
warnings.filterwarnings('ignore')

In [ ]:

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/Liver Disease Prediction/train_dataset.csv")
test_df = pd.read_csv("/content/drive/MyDrive/Liver Disease Prediction/test_dataset.csv")
train_df.head(15)

,ID,N_Days,Status,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,7135,1654,CL,D-penicillamine,19581,F,N,N,Y,N,0.3,279.0,2.96,84.0,1500.8,99.43,109.0,293.0,10.2,4.0
1,7326,41,C,D-penicillamine,22880,F,NaN,N,NaN,N,0.3,NaN,2.96,NaN,1835.4,26.35,131.0,308.0,10.8,1.0
2,7254,297,D,NaN,27957,F,N,N,NaN,N,0.3,328.0,2.64,4.0,NaN,NaN,116.0,194.0,10.3,3.0
3,3135,1872,C,D-penicillamine,21111,F,NaN,Y,Y,N,0.3,302.0,2.02,49.0,NaN,26.35,NaN,NaN,10.5,4.0
4,2483,939,CL,D-penicillamine,18061,F,NaN,NaN,NaN,N,0.5,344.0,3.11,91.0,NaN,104.56,NaN,306.0,11.4,2.0
5,2306,1255,C,Placebo,14353,F,Y,N,N,N,9.9,NaN,3.57,4.0,462.6,103.40,126.0,276.0,12.1,4.0
6,2525,2065,C,NaN,25357,F,N,Y,NaN,N,0.3,NaN,2.60,103.0,NaN,NaN,200.0,365.0,10.5,4.0
7,6908,1118,D,Placebo,28620,F,NaN,NaN,N,N,2.0,357.0,3.11,94.0,3403.8,NaN,NaN,254.0,10.1,4.0
8,8388,4313,D,NaN,24638,F,N,N,NaN,N,1.2,NaN,3.51,67.0,1850.6,44.21,NaN,434.0,11.1,2.0
9,9132,3240,D,NaN,13860,F,N,N,NaN,N,8.7,NaN,3.02,NaN,NaN,87.62,NaN,102.0,11.2,3.0


In [ ]:
print("============= TRAIN SET =============")
print("Missing Values: " ,np.sum(train_df.isnull().any(axis=1)))
print("Shape: ", train_df.shape)
print("Columns: ", len(train_df.columns))
print("Data types:")
print(train_df.dtypes)
print("============= TEST SET =============")
print("Missing Values: ",(np.sum(test_df.isnull().any(axis=1))))
print("Shape: ", test_df.shape)
print("Columns: ", len(test_df.columns))
print("Data types:")
print(test_df.dtypes)

============= TRAIN SET =============
Missing Values:  6685
Shape:  (6800, 20)
Columns:  20
Data types:
ID                 int64
N_Days             int64
Status            object
Drug              object
Age                int64
Sex               object
Ascites           object
Hepatomegaly      object
Spiders           object
Edema             object
Bilirubin        float64
Cholesterol      float64
Albumin          float64
Copper           float64
Alk_Phos         float64
SGOT             float64
Tryglicerides    float64
Platelets        float64
Prothrombin      float64
Stage            float64
dtype: object
============= TEST SET =============
Missing Values:  3136
Shape:  (3200, 19)
Columns:  19
Data types:
ID                 int64
N_Days             int64
Status            object
Drug              object
Age                int64
Sex               object
Ascites           object
Hepatomegaly      object
Spiders           object
Edema             object
Bilirubin        float64
Chol

In [ ]:
train_df.isnull()

,ID,N_Days,Status,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,True,False,True,False,False,True,False,True,False,False,False,False,False,False
2,False,False,False,True,False,False,False,False,True,False,False,False,False,False,True,True,False,False,False,False
3,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,True,True,False,False
4,False,False,False,False,False,False,True,True,True,False,False,False,False,False,True,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6795,False,False,False,False,False,False,True,False,True,False,False,False,False,True,False,False,False,False,False,False
6796,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,True,False,False,False,False
6797,False,False,False,True,False,False,False,False,True,False,False,False,False,True,True,False,False,False,False,False
6798,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False


In [ ]:
test_df.isnull()

,ID,N_Days,Status,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin
0,False,False,False,False,False,False,False,True,False,False,False,False,False,True,True,False,True,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,True,False,True,True,False,True,False,False
3,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,True,False,False
4,False,False,False,False,False,False,True,True,True,False,False,True,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,False,False,False,False,False,False,True,False,False,False,False,True,False,True,False,False,True,False,False
3196,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
3197,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False
3198,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
train_df.describe()

,ID,N_Days,Age,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
count,6800.000000,6800.000000,6800.000000,6800.000000,3699.000000,6800.000000,4644.000000,4302.000000,4698.000000,3988.000000,6462.000000,6645.000000,6800.000000
mean,6268.608088,1664.055441,22863.223529,1.806147,294.575831,3.036118,68.569121,1802.951464,79.554427,133.857823,293.306097,11.231031,3.157206
std,2405.527133,1332.308135,4671.135682,3.218009,128.085317,0.528164,59.637733,1618.550339,43.433832,47.291444,115.146088,1.123444,0.993059
min,1239.000000,41.000000,9598.000000,0.300000,120.000000,1.960000,4.000000,289.000000,26.350000,33.000000,62.000000,9.000000,1.000000
25%,4353.750000,537.000000,19416.500000,0.300000,204.000000,2.680000,26.000000,1122.650000,35.290000,114.000000,213.000000,10.500000,2.000000
50%,6599.500000,1443.000000,23367.000000,0.400000,312.000000,3.020000,68.000000,1563.150000,88.505000,128.000000,285.000000,11.100000,4.000000
75%,8299.250000,2519.000000,27125.500000,1.800000,353.000000,3.390000,91.000000,2013.250000,110.672500,141.000000,365.000000,11.700000,4.000000
max,9999.000000,4795.000000,28650.000000,28.000000,1775.000000,4.640000,588.000000,13862.400000,340.480000,498.000000,563.000000,18.000000,4.000000


In [ ]:
train_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6800 entries, 0 to 6799
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID             6800 non-null   int64  
 1   N_Days         6800 non-null   int64  
 2   Status         6800 non-null   object 
 3   Drug           4775 non-null   object 
 4   Age            6800 non-null   int64  
 5   Sex            6800 non-null   object 
 6   Ascites        4554 non-null   object 
 7   Hepatomegaly   4373 non-null   object 
 8   Spiders        4210 non-null   object 
 9   Edema          6800 non-null   object 
 10  Bilirubin      6800 non-null   float64
 11  Cholesterol    3699 non-null   float64
 12  Albumin        6800 non-null   float64
 13  Copper         4644 non-null   float64
 14  Alk_Phos       4302 non-null   float64
 15  SGOT           4698 non-null   float64
 16  Tryglicerides  3988 non-null   float64
 17  Platelets      6462 non-null   float64
 18  Prothrom

In [ ]:
duplicate_train_df = train_df[train_df.duplicated()]
duplicate_train_df.shape

(0, 20)

In [ ]:
duplicate_test_df = test_df[test_df.duplicated()]
duplicate_test_df.shape

(0, 19)

In [ ]:
percent_missing1 = round((train_df.isnull().mean() * 100),2)
percent_missing2 = round((test_df.isnull().mean() * 100),2)


In [ ]:
missing_value_df1 = pd.DataFrame({'column_name': train_df.columns,
                                 'percent_missing': percent_missing1})

In [ ]:
missing_value_df2 = pd.DataFrame({'column_name': test_df.columns,
                                 'percent_missing': percent_missing2})

In [ ]:
missing_value_df1

,column_name,percent_missing
ID,ID,0.00
N_Days,N_Days,0.00
Status,Status,0.00
Drug,Drug,29.78
Age,Age,0.00
Sex,Sex,0.00
Ascites,Ascites,33.03
Hepatomegaly,Hepatomegaly,35.69
Spiders,Spiders,38.09
Edema,Edema,0.00


In [ ]:
missing_value_df2

,column_name,percent_missing
ID,ID,0.00
N_Days,N_Days,0.00
Status,Status,0.00
Drug,Drug,30.09
Age,Age,0.00
Sex,Sex,0.00
Ascites,Ascites,33.97
Hepatomegaly,Hepatomegaly,34.44
Spiders,Spiders,37.50
Edema,Edema,0.00


In [ ]:
test_df = test_df.fillna(method = 'bfill')

In [ ]:
test_df.dropna()

,ID,N_Days,Status,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin
0,3870,41,C,Placebo,22553,F,N,Y,N,N,1.4,247.0,3.62,92.0,1748.1,108.65,129.0,169.0,11.6
1,3462,1811,C,D-penicillamine,16223,F,N,Y,N,N,0.3,311.0,2.80,92.0,1748.1,43.52,129.0,321.0,11.5
2,1632,954,C,D-penicillamine,27100,F,N,N,N,N,0.4,206.0,3.56,89.0,617.1,43.52,129.0,296.0,10.3
3,722,1969,D,Placebo,17039,F,N,Y,N,N,1.2,206.0,3.16,89.0,617.1,113.76,129.0,125.0,10.9
4,1000,2721,D,D-penicillamine,17738,F,N,N,N,N,3.2,206.0,2.36,89.0,1782.4,37.87,129.0,138.0,10.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3194,4118,4324,C,D-penicillamine,18692,F,Y,N,N,N,0.3,120.0,2.21,47.0,1805.0,111.68,166.0,207.0,9.7
3195,2601,4513,D,Placebo,18293,M,N,N,N,S,0.3,422.0,2.75,106.0,2063.9,111.68,166.0,563.0,10.9
3196,53,3281,D,D-penicillamine,12149,F,N,N,Y,N,4.7,422.0,4.30,106.0,1203.2,32.49,166.0,234.0,10.2
3197,386,1933,D,D-penicillamine,17084,F,N,Y,N,N,0.8,408.0,3.56,9.0,2358.1,124.70,131.0,234.0,11.6


In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3200 entries, 0 to 3199
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID             3200 non-null   int64  
 1   N_Days         3200 non-null   int64  
 2   Status         3200 non-null   object 
 3   Drug           3200 non-null   object 
 4   Age            3200 non-null   int64  
 5   Sex            3200 non-null   object 
 6   Ascites        3200 non-null   object 
 7   Hepatomegaly   3200 non-null   object 
 8   Spiders        3200 non-null   object 
 9   Edema          3200 non-null   object 
 10  Bilirubin      3200 non-null   float64
 11  Cholesterol    3199 non-null   float64
 12  Albumin        3200 non-null   float64
 13  Copper         3200 non-null   float64
 14  Alk_Phos       3200 non-null   float64
 15  SGOT           3199 non-null   float64
 16  Tryglicerides  3200 non-null   float64
 17  Platelets      3200 non-null   float64
 18  Prothrom

In [ ]:
le = LabelEncoder()

In [ ]:
train_df.Status = le.fit_transform(train_df.Status)
train_df.Drug = le.fit_transform(train_df.Drug)
train_df.Sex = le.fit_transform(train_df.Sex)
train_df.Cholesterol = le.fit_transform(train_df.Cholesterol)
train_df.Ascites = le.fit_transform(train_df.Ascites)
train_df.Hepatomegaly = le.fit_transform(train_df.Hepatomegaly)
train_df.Spiders = le.fit_transform(train_df.Spiders)
train_df.Edema = le.fit_transform(train_df.Edema)
train_df.Albumin = le.fit_transform(train_df.Albumin)
train_df.Copper= le.fit_transform(train_df.Copper)
train_df.Alk_Phos= le.fit_transform(train_df.Alk_Phos)
train_df.SGOT= le.fit_transform(train_df.SGOT)
train_df.Tryglicerides= le.fit_transform(train_df.Tryglicerides)
train_df.Platelets = le.fit_transform(train_df.Platelets)
train_df.Prothrombin = le.fit_transform(train_df.Prothrombin)


In [ ]:
test_df.Status = le.fit_transform(test_df.Status)
test_df.Drug = le.fit_transform(test_df.Drug)
test_df.Sex = le.fit_transform(test_df.Sex)
test_df.Ascites = le.fit_transform(test_df.Ascites)
test_df.Hepatomegaly = le.fit_transform(test_df.Hepatomegaly)
test_df.Spiders = le.fit_transform(test_df.Spiders)
test_df.Edema = le.fit_transform(test_df.Edema)
test_df.Albumin = le.fit_transform(test_df.Albumin)
test_df.Copper= le.fit_transform(test_df.Copper)
test_df.Alk_Phos= le.fit_transform(test_df.Alk_Phos)
test_df.SGOT= le.fit_transform(test_df.SGOT)
test_df.Tryglicerides= le.fit_transform(test_df.Tryglicerides)
test_df.Platelets = le.fit_transform(test_df.Platelets)
test_df.Prothrombin = le.fit_transform(test_df.Prothrombin)


In [ ]:
train_df['Age']= train_df['Age'].astype('int64')
train_df['Bilirubin']= train_df['Bilirubin'].astype('int64')
train_df['Cholesterol']= train_df['Cholesterol'].astype('int64')
train_df['Albumin']= train_df['Albumin'].astype('int64')
train_df['Copper']= train_df['Copper'].astype('int64')
train_df['Alk_Phos']= train_df['Alk_Phos'].astype('int64')
train_df['SGOT']= train_df['SGOT'].astype('int64')
train_df['Tryglicerides']= train_df['Tryglicerides'].astype('int64')
train_df['Platelets']= train_df['Platelets'].astype('int64')
train_df['Prothrombin']= train_df['Prothrombin'].astype('int64')




test_df['Age']= test_df['Age'].astype('int64')
test_df['Bilirubin']= test_df['Bilirubin'].astype('int64')
test_df['Cholesterol']= test_df['Cholesterol'].astype('Int64')
test_df['Albumin']= test_df['Albumin'].astype('int64')
test_df['Copper']= test_df['Copper'].astype('int64')
test_df['Alk_Phos']= test_df['Alk_Phos'].astype('int64')
test_df['SGOT']= test_df['SGOT'].astype('int64')
test_df['Tryglicerides']= test_df['Tryglicerides'].astype('int64')
test_df['Platelets']= test_df['Platelets'].astype('int64')
test_df['Prothrombin']= test_df['Prothrombin'].astype('int64')


In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3200 entries, 0 to 3199
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   ID             3200 non-null   int64
 1   N_Days         3200 non-null   int64
 2   Status         3200 non-null   int64
 3   Drug           3200 non-null   int64
 4   Age            3200 non-null   int64
 5   Sex            3200 non-null   int64
 6   Ascites        3200 non-null   int64
 7   Hepatomegaly   3200 non-null   int64
 8   Spiders        3200 non-null   int64
 9   Edema          3200 non-null   int64
 10  Bilirubin      3200 non-null   int64
 11  Cholesterol    3199 non-null   Int64
 12  Albumin        3200 non-null   int64
 13  Copper         3200 non-null   int64
 14  Alk_Phos       3200 non-null   int64
 15  SGOT           3200 non-null   int64
 16  Tryglicerides  3200 non-null   int64
 17  Platelets      3200 non-null   int64
 18  Prothrombin    3200 non-null   int64
dtypes: Int

In [ ]:
x_1 = train_df.loc[train_df['Stage'] == 1.0]
x_2 = train_df.loc[train_df['Stage'] == 2.0]
x_3 = train_df.loc[train_df['Stage'] == 3.0]
x_4 = train_df.loc[train_df['Stage'] == 4.0]

In [ ]:
x_1

,ID,N_Days,Status,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
1,7326,41,0,0,22880,0,2,0,2,0,0,456,100,277,2246,0,98,245,18,1.0
31,7577,1343,2,0,20401,1,2,1,0,0,1,456,154,277,3607,2054,78,215,9,1.0
47,6760,4658,0,2,22616,0,2,2,1,0,0,37,155,97,670,1490,100,41,23,1.0
51,4312,2762,2,1,28650,0,2,2,1,0,0,210,111,98,677,2842,212,498,10,1.0
77,6055,41,2,2,28650,0,1,1,2,0,0,456,101,54,1268,3129,268,142,30,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6763,5668,152,0,1,28650,0,0,2,0,0,0,456,166,277,3607,1654,74,305,46,1.0
6771,9807,2944,2,2,20020,0,2,2,0,0,0,456,65,277,103,1100,164,497,5,1.0
6785,4175,2578,2,1,26068,0,0,0,0,0,0,0,170,37,3607,0,77,497,27,1.0
6795,4622,1762,0,1,20909,1,2,0,2,0,0,240,47,277,3341,2869,97,251,39,1.0


In [ ]:
x_2_1 = x_2.iloc[:753]
x_2_2 = x_2.iloc[754:1507]
print(len(x_2_1),len(x_2_2))

753 753


In [ ]:
x_3_1 = x_3.iloc[:660]
x_3_2 = x_3.iloc[661:1322]
print(len(x_3_1),len(x_3_2))

660 661


In [ ]:
x_4_1 = x_4.iloc[:875]
x_4_2 = x_4.iloc[876:1751]
x_4_3 = x_4.iloc[1752:2627]
x_4_4 = x_4.iloc[2628:3504]
print(len(x_4_1),len(x_4_2),len(x_4_3),len(x_4_4))

875 875 875 876


In [ ]:
x_1 = pd.concat([x_1,x_2_1,x_3_1,x_4_1])
x_2 = pd.concat([x_1,x_2_1,x_3_1,x_4_2])
x_3 = pd.concat([x_1,x_2_1,x_3_1,x_4_2])
x_4 = pd.concat([x_1,x_2_1,x_3_1,x_4_4])
x_5 = pd.concat([x_1,x_2_2,x_3_1,x_4_1])
x_6 = pd.concat([x_1,x_2_2,x_3_1,x_4_2])
x_7 = pd.concat([x_1,x_2_2,x_3_1,x_4_3])
x_8 = pd.concat([x_1,x_2_2,x_3_1,x_4_4])

In [ ]:
x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8

(        ID  N_Days  Status  Drug  ...  Tryglicerides  Platelets  Prothrombin  Stage
 1     7326      41       0     0  ...             98        245           18    1.0
 31    7577    1343       2     0  ...             78        215            9    1.0
 47    6760    4658       0     2  ...            100         41           23    1.0
 51    4312    2762       2     1  ...            212        498           10    1.0
 77    6055      41       2     2  ...            268        142           30    1.0
 ...    ...     ...     ...   ...  ...            ...        ...          ...    ...
 1665  1412    1310       1     0  ...            268        109            9    4.0
 1666  7009    2164       2     2  ...             88        462           21    4.0
 1668  5775    2185       0     0  ...             95        397           19    4.0
 1673  2958      41       0     0  ...             91        182           64    4.0
 1674  2473      41       0     2  ...            268        497 

In [ ]:
x_1 = x_1.sample(frac=1).reset_index(drop=True)
x_2 = x_2.sample(frac=1).reset_index(drop=True)
x_3 = x_3.sample(frac=1).reset_index(drop=True)
x_4 = x_4.sample(frac=1).reset_index(drop=True)
x_5 = x_5.sample(frac=1).reset_index(drop=True)
x_6 = x_6.sample(frac=1).reset_index(drop=True)
x_7 = x_7.sample(frac=1).reset_index(drop=True)
x_8 = x_8.sample(frac=1).reset_index(drop=True)

In [ ]:
x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8

(        ID  N_Days  Status  Drug  ...  Tryglicerides  Platelets  Prothrombin  Stage
 0     2473      41       0     2  ...            268        497           13    4.0
 1     2820     332       2     0  ...             79        363           21    4.0
 2     7941     112       2     2  ...             87        278           23    3.0
 3     4832     890       0     1  ...             87        177           20    2.0
 4     7589    2587       0     2  ...            268        289           15    2.0
 ...    ...     ...     ...   ...  ...            ...        ...          ...    ...
 2748  9197     166       0     2  ...             89        191           16    2.0
 2749  5573    2997       2     2  ...              0        253           11    4.0
 2750  6062    1958       0     2  ...            268        301           21    4.0
 2751  7366     989       2     2  ...            268        366           52    3.0
 2752  4265    3781       2     0  ...            268        285 

In [ ]:
x_1.info(),x_2.info(),x_3.info(),x_4.info(),x_5.info(),x_6.info(),x_7.info(),x_8.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2753 entries, 0 to 2752
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID             2753 non-null   int64  
 1   N_Days         2753 non-null   int64  
 2   Status         2753 non-null   int64  
 3   Drug           2753 non-null   int64  
 4   Age            2753 non-null   int64  
 5   Sex            2753 non-null   int64  
 6   Ascites        2753 non-null   int64  
 7   Hepatomegaly   2753 non-null   int64  
 8   Spiders        2753 non-null   int64  
 9   Edema          2753 non-null   int64  
 10  Bilirubin      2753 non-null   int64  
 11  Cholesterol    2753 non-null   int64  
 12  Albumin        2753 non-null   int64  
 13  Copper         2753 non-null   int64  
 14  Alk_Phos       2753 non-null   int64  
 15  SGOT           2753 non-null   int64  
 16  Tryglicerides  2753 non-null   int64  
 17  Platelets      2753 non-null   int64  
 18  Prothrom

(None, None, None, None, None, None, None, None)

In [ ]:
x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8

(        ID  N_Days  Status  Drug  ...  Tryglicerides  Platelets  Prothrombin  Stage
 0     2473      41       0     2  ...            268        497           13    4.0
 1     2820     332       2     0  ...             79        363           21    4.0
 2     7941     112       2     2  ...             87        278           23    3.0
 3     4832     890       0     1  ...             87        177           20    2.0
 4     7589    2587       0     2  ...            268        289           15    2.0
 ...    ...     ...     ...   ...  ...            ...        ...          ...    ...
 2748  9197     166       0     2  ...             89        191           16    2.0
 2749  5573    2997       2     2  ...              0        253           11    4.0
 2750  6062    1958       0     2  ...            268        301           21    4.0
 2751  7366     989       2     2  ...            268        366           52    3.0
 2752  4265    3781       2     0  ...            268        285 

In [ ]:
x_1.info()
x_2.info()
x_3.info()
x_4.info()
x_5.info()
x_6.info()
x_7.info()
x_8.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2753 entries, 0 to 2752
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID             2753 non-null   int64  
 1   N_Days         2753 non-null   int64  
 2   Status         2753 non-null   int64  
 3   Drug           2753 non-null   int64  
 4   Age            2753 non-null   int64  
 5   Sex            2753 non-null   int64  
 6   Ascites        2753 non-null   int64  
 7   Hepatomegaly   2753 non-null   int64  
 8   Spiders        2753 non-null   int64  
 9   Edema          2753 non-null   int64  
 10  Bilirubin      2753 non-null   int64  
 11  Cholesterol    2753 non-null   int64  
 12  Albumin        2753 non-null   int64  
 13  Copper         2753 non-null   int64  
 14  Alk_Phos       2753 non-null   int64  
 15  SGOT           2753 non-null   int64  
 16  Tryglicerides  2753 non-null   int64  
 17  Platelets      2753 non-null   int64  
 18  Prothrom

In [ ]:
y_1 = x_1['Stage']
x_1 = x_1.drop(['Stage'],axis=1)

y_2 = x_2['Stage']
x_2 = x_2.drop(['Stage'],axis=1)

y_3 = x_3['Stage']
x_3 = x_3.drop(['Stage'],axis=1)

y_4 = x_4['Stage']
x_4 = x_4.drop(['Stage'],axis=1)

y_5 = x_5['Stage']
x_5 = x_5.drop(['Stage'],axis=1)

y_6 = x_6['Stage']
x_6 = x_6.drop(['Stage'],axis=1)

y_7 = x_7['Stage']
x_7 = x_7.drop(['Stage'],axis=1)

y_8 = x_8['Stage']
x_8 = x_8.drop(['Stage'],axis=1)



In [ ]:
X_1_train, X_1_test, y_1_train, y_1_test = train_test_split(x_1, y_1, test_size=0.1, random_state=2, shuffle=True)
X_2_train, X_2_test, y_2_train, y_2_test = train_test_split(x_2, y_2, test_size=0.1, random_state=2, shuffle=True)
X_3_train, X_3_test, y_3_train, y_3_test = train_test_split(x_3, y_3, test_size=0.1, random_state=2, shuffle=True)
X_4_train, X_4_test, y_4_train, y_4_test = train_test_split(x_4, y_4, test_size=0.1, random_state=2, shuffle=True)
X_5_train, X_5_test, y_5_train, y_5_test = train_test_split(x_5, y_5, test_size=0.1, random_state=2, shuffle=True)
X_6_train, X_6_test, y_6_train, y_6_test = train_test_split(x_6, y_6, test_size=0.1, random_state=2, shuffle=True)
X_7_train, X_7_test, y_7_train, y_7_test = train_test_split(x_7, y_7, test_size=0.1, random_state=2, shuffle=True)
X_8_train, X_8_test, y_8_train, y_8_test = train_test_split(x_8, y_8, test_size=0.1, random_state=2, shuffle=True)

In [ ]:
clf1 = tree.DecisionTreeClassifier(max_depth=32)

In [ ]:
cvs_1 = (np.mean(cross_val_score(clf1, X_1_train, y_1_train, cv=10)))
cvs_2 = (np.mean(cross_val_score(clf1, X_2_train, y_2_train, cv=10)))
cvs_3 = (np.mean(cross_val_score(clf1, X_3_train, y_3_train, cv=10)))
cvs_4 = (np.mean(cross_val_score(clf1, X_4_train, y_4_train, cv=10)))
cvs_5 = (np.mean(cross_val_score(clf1, X_5_train, y_5_train, cv=10)))
cvs_6 = (np.mean(cross_val_score(clf1, X_6_train, y_6_train, cv=10)))
cvs_7 = (np.mean(cross_val_score(clf1, X_7_train, y_7_train, cv=10)))
cvs_8 = (np.mean(cross_val_score(clf1, X_8_train, y_8_train, cv=10)))       

In [ ]:
cvs_1,cvs_2,cvs_3,cvs_4,cvs_5,cvs_6,cvs_7,cvs_8

(0.347616560010448,
 0.6873987416246073,
 0.6809887096303644,
 0.6916406531104433,
 0.6818639320827377,
 0.5097028133539497,
 0.5160924234909705,
 0.5003272359502484)

In [ ]:
clf1.fit(X_1_train,y_1_train)
clf1.fit(X_2_train,y_2_train)
clf1.fit(X_3_train,y_3_train)
clf1.fit(X_4_train,y_4_train)
clf1.fit(X_5_train,y_5_train)
clf1.fit(X_6_train,y_6_train)
clf1.fit(X_7_train,y_7_train)
clf1.fit(X_8_train,y_8_train)

DecisionTreeClassifier(max_depth=32)

In [ ]:
print(clf1.score(X_1_test,y_1_test))
print(clf1.score(X_2_test,y_2_test))
print(clf1.score(X_3_test,y_3_test))
print(clf1.score(X_4_test,y_4_test))
print(clf1.score(X_5_test,y_5_test))
print(clf1.score(X_6_test,y_6_test))
print(clf1.score(X_7_test,y_7_test))
print(clf1.score(X_8_test,y_8_test))

0.9456521739130435
0.8673267326732673
0.8673267326732673
0.9386138613861386
0.9584158415841584
0.8712871287128713
0.8475247524752475
0.5326732673267327


In [ ]:
y_pred = clf1.predict(X_5_test)
y_preds = np.round(y_pred)
print('Accuracy Score is: ', accuracy_score(y_5_test,y_preds))

Accuracy Score is:  0.9584158415841584


In [ ]:
Y_1 = pd.concat([y_1_test,y_2_test,y_3_test,y_4_test,
                 y_5_test,y_6_test,y_7_test])

In [ ]:
Y_1

1743    3.0
2705    1.0
642     4.0
2012    2.0
79      2.0
       ... 
368     4.0
2703    3.0
1439    4.0
1746    4.0
862     4.0
Name: Stage, Length: 3306, dtype: float64

In [ ]:
cm = confusion_matrix(y_5_test,y_preds)

In [ ]:
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
print(classification_report(y_5_test,y_preds))

              precision    recall  f1-score   support

         1.0       0.96      0.90      0.93        52
         2.0       0.99      0.93      0.96       165
         3.0       0.94      0.99      0.97       144
         4.0       0.95      0.97      0.96       144

    accuracy                           0.96       505
   macro avg       0.96      0.95      0.95       505
weighted avg       0.96      0.96      0.96       505



In [ ]:
Y_1

In [ ]:
Y_5_test = X_5_test['ID']
X_5_test = X_5_test.drop(columns = ['ID'])


In [ ]:
Y_5_test

In [ ]:
y_preds

In [ ]:
df_target=pd.DataFrame(Y_1)

 # its important for comparison. Here "test_new" is your new test dataset
df_target.columns = ['Stage']
df_target

,Stage
1743,3.0
2705,1.0
642,4.0
2012,2.0
79,2.0
...,...
368,4.0
2703,3.0
1439,4.0
1746,4.0


In [ ]:

df_target.to_csv('prediction_results.csv', index = True)         
